In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import tensorflow_hub as hub
from datetime import datetime
from scipy.spatial.distance import cosine
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
person = None
people_sentences = dict()
sentence_list = list()
people_list = list()
i=0
with open("/content/drive/My Drive/NLP Data.txt", 'r', encoding='utf8') as handle:
    sentences = handle.readlines()
    for sentence in sentences[2:]:
        i += 1
        sentence = sentence.strip()
        if len(sentence)<15:
            continue
        if '#@#@#' in sentence:
            sentence = sentence.strip(' \n@#')
            person = sentence.lower().strip()
        else:
            sentence = sentence.lower()
            if person in sentence:
                sentence = sentence.replace(person, 'x')

            sentence_list.append(sentence)
            people_list.append(person)


In [0]:
with open("/content/drive/My Drive/random.txt", 'r', encoding='utf8') as handle:
    clean_sentences = list()
    sentences = handle.readlines()
    for sentence in sentences:
        sentence = sentence.strip()
        if len(sentence)<15:
            continue
        clean_sentences.append(sentence)
    from random import sample 
    clean_sentences = sample(clean_sentences, 4500)
    sentence_list += clean_sentences
    people_list += ['None']*4500

In [0]:
people_sentences['Sentence'] = sentence_list
people_sentences['Person'] = people_list
data = pd.DataFrame.from_dict(people_sentences)

In [18]:
data['Person'].value_counts()

None         4470
jackson      2071
dicaprio     1926
tendulkar    1164
federer      1033
gates         983
beyonce       890
bengio         36
Name: Person, dtype: int64

In [0]:
from sklearn.utils import shuffle
data = shuffle(data)
data.Person = data.Person.astype('category')

test_data = pd.DataFrame()
for person in data['Person'].unique():
    test_data = test_data.append(data.loc[data['Person'] == person].sample(30))

data = data[~data.Sentence.isin(test_data.Sentence)].dropna()
train_text = data['Sentence'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = np.asarray(pd.get_dummies(data.Person), dtype = np.int8)

test_data = shuffle(test_data)
test_data.Person = test_data.Person.astype('category')
test_text = test_data['Sentence'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = np.asarray(pd.get_dummies(test_data.Person), dtype = np.int8)

In [19]:
data.shape

(12573, 2)

In [0]:
# Used for Universal Sentence Encoder sentence embedding generation
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [0]:
# Used for Universal Sentence Encoder sentence embedding generation
def GenUniversalSentenceEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
    	signature="default", as_dict=True)["default"]

In [11]:
# Used for BERT sentence embedding generation
!pip install pytorch-pretrained-bert
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM


     |████████████████████████████████| 133kB 5.9MB/s 
     |████████████████████████████████| 675kB 11.4MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609175 sha256=075fc6ecf4ceb9d2e97a8a334a9c62125e2f27f5992d0f3513a1481b3ffd9c0b
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built regex


In [0]:
# Used for BERT sentence embedding generation
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 2667901.95B/s]


In [0]:
# Used for BERT sentence embedding generation
# @tf.function
def BertSentenceEmbedding(text):
    marked_text = "[CLS] " + text + " [SEP]"
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)
    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Mark each of the 22 tokens as belonging to sentence "1".
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    
    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(encoded_layers, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    # `encoded_layers` has shape [12 x 1 x 22 x 768]
    # `token_vecs` is a tensor with shape [22 x 768]
    token_vecs = encoded_layers[11][0]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    return sentence_embedding

In [0]:
# Used for BERT sentence embedding generation
def return_bert_embeddings_for_text(input_text):
    train_embeddings = []
    for text in input_text:
        text = text[0]
        embedding = BertSentenceEmbedding(text)
        train_embeddings.append(embedding)
    return train_embeddings

In [0]:
# Used for BERT sentence embedding generation
train_embeddings = return_bert_embeddings_for_text(train_text)
train_embeddings = np.array(train_embeddings, dtype=object)[:, np.newaxis]
validation_embeddings = return_bert_embeddings_for_text(test_text)
validation_embeddings = np.array(validation_embeddings, dtype=object)[:, np.newaxis]

In [0]:
# Used for BERT sentence embedding generation
train_embeddings.shape

(12609, 1)

In [25]:
from tensorflow.keras.models import Model
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(GenUniversalSentenceEmbedding,
	output_shape=(None,))(input_text)
dense = layers.Dense(300, activation='relu')(embedding)
dense = layers.BatchNormalization()(dense)
dense = layers.Dropout(0.4)(dense)
dense1 = layers.Dense(200, activation='relu')(dense)
dense1 = layers.BatchNormalization()(dense1)
dense1 = layers.Dropout(0.4)(dense1)
dense2 = layers.Dense(100, activation='relu')(dense1)
pred = layers.Dense(len(data['Person'].unique()), activation='softmax')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', 
	optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [26]:
from keras import backend as K
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=15,
            batch_size=32)
  model.save_weights('./model.h6')

Train on 12573 samples, validate on 240 samples
Epoch 1/15
12573/12573 [==============================] - 20s 2ms/sample - loss: 0.6353 - acc: 0.7825 - val_loss: 1.0645 - val_acc: 0.7417
Epoch 2/15
12573/12573 [==============================] - 17s 1ms/sample - loss: 0.4430 - acc: 0.8421 - val_loss: 0.5207 - val_acc: 0.8333
Epoch 3/15
12573/12573 [==============================] - 17s 1ms/sample - loss: 0.3831 - acc: 0.8593 - val_loss: 0.5706 - val_acc: 0.8125
Epoch 4/15
12573/12573 [==============================] - 17s 1ms/sample - loss: 0.3550 - acc: 0.8665 - val_loss: 0.4740 - val_acc: 0.8500
Epoch 5/15
12573/12573 [==============================] - 17s 1ms/sample - loss: 0.3225 - acc: 0.8768 - val_loss: 0.4201 - val_acc: 0.8667
Epoch 6/15
12573/12573 [==============================] - 17s 1ms/sample - loss: 0.3136 - acc: 0.8788 - val_loss: 0.4023 - val_acc: 0.8708
Epoch 7/15
12573/12573 [==============================] - 17s 1ms/sample - loss: 0.3011 - acc: 0.8876 - val_loss: 0.44

In [0]:
# Used for BERT sentence embedding generation
train_embeddings_m = []
for embedding in train_embeddings:
    train_embeddings_m.append(embedding[0].tolist())
validation_embeddings_m = []
for embedding in validation_embeddings:
    validation_embeddings_m.append(embedding[0].tolist())
train_embeddings_m = np.array(train_embeddings_m)
validation_embeddings_m = np.array(validation_embeddings_m)
train_embeddings_m.shape

In [0]:
new_text = ["Who is born in Switzerland ?", 
            "Who is a singer ?", 
            "Who is a dancer ?",
            "Whose album was on BillBoard ?",
            "Whose won Turing award ?",
            "Whose is working in artificial intelligence ?",
            "Whose is working in machine learning ?",
            "Who is a Cricketer ?",
            "Who belongs to India ?",
            "Who is Rich ?",
            "Who served Prison Time ?",
            "Who is in Software Industry ?",
            "Who is not a singer ?",
            "Who is a Computer Scientist ?",
            "Who is Scientist ?",
            "Who is into the Research ?",
            "Who sponsored the brand MRF ?",
            "Who does farming ?",
            "Who owns a vineyard ?",
            "Who is famous ?",
            "Who is Corrupted ?",
            "Who is an alcoholic ?",
            "Who grew up in Namibia ?",
            "Who owns the military ?",
            "Who is She ?",
            "Who is Woman ?",
            "Who is Female ?",
            "Who plays Tennis ?",
            "Who is a Movie Star ?",
            "Who is divorced ?",
            "Who is dead ?",
            "Who plays video games ?",
            "Who played in Rawalpindi ?",
            "Who is born in Pakistan ?",
            "Who won Grand Slams ?",
            "Who is called Master Blaster ?",
            "Who is member of Rajya Sabha ?",
            "Who worked with Scorsese ?",
            "Who worked with Director Scorsese ?",
            "Who worked with Movie Director Scorsese ?",
            "Who is an enviromentalist ?"]
new_text = np.array(new_text, dtype=object)[:, np.newaxis]
# test_embeddings = return_bert_embeddings_for_text(new_text)
# test_embeddings = np.array(test_embeddings, dtype=object)[:, np.newaxis]


In [0]:
# Used for BERT sentence embedding generation
test_embeddings_m = []
for embedding in test_embeddings:
    test_embeddings_m.append(embedding[0].tolist())
test_embeddings_m = np.array(test_embeddings_m)

In [31]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h6')  
  predicts = model.predict(new_text, batch_size=32)

categories = data.Person.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
for i in range(len(predict_labels)):
    print(new_text[i], predict_labels[i])

['Who is born in Switzerland ?'] federer
['Who is a singer ?'] beyonce
['Who is a dancer ?'] jackson
['Whose album was on BillBoard ?'] jackson
['Whose won Turing award ?'] bengio
['Whose is working in artificial intelligence ?'] bengio
['Whose is working in machine learning ?'] bengio
['Who is a Cricketer ?'] tendulkar
['Who belongs to India ?'] None
['Who is Rich ?'] gates
['Who served Prison Time ?'] jackson
['Who is in Software Industry ?'] gates
['Who is not a singer ?'] jackson
['Who is a Computer Scientist ?'] gates
['Who is Scientist ?'] dicaprio
['Who is into the Research ?'] gates
['Who sponsored the brand MRF ?'] tendulkar
['Who does farming ?'] None
['Who owns a vineyard ?'] None
['Who is famous ?'] dicaprio
['Who is Corrupted ?'] None
['Who is an alcoholic ?'] None
['Who grew up in Namibia ?'] federer
['Who owns the military ?'] dicaprio
['Who is She ?'] beyonce
['Who is Woman ?'] None
['Who is Female ?'] None
['Who plays Tennis ?'] federer
['Who is a Movie Star ?'] dicapr